In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from google.colab import drive
import yfinance as yf
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout, Flatten, Bidirectional
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from math import sqrt
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Volatility/New_data_2000_2024/Just_normal/'

Mounted at /content/drive


In [ ]:
def set_index(df):
  df.index = pd.to_datetime(df['Date'])
  df.drop(columns=['Date'], inplace=True)

In [ ]:
filename = 'RSV_data.csv'
RSV_data = pd.read_csv(path + filename)

In [ ]:
set_index(RSV_data)

In [ ]:
data = pd.DataFrame(columns=['garch', 'return'], index=RSV_data.index)

In [ ]:
data['garch'] = RSV_data["('GARCH', 'normal', 0)"].values

In [ ]:
data['return'] = RSV_data["log_returns"].values

In [ ]:
data['RSV'] = RSV_data["RSV"].values

In [ ]:
data

,garch,return,RSV
Date,,,
2000-02-02,0.800313,-0.027564,0.889609
2000-02-03,0.655577,0.893688,0.929549
2000-02-04,0.491617,-0.052641,0.802329
2000-02-07,0.408321,-0.025746,0.785662
2000-02-08,0.339040,0.976294,0.760232
...,...,...,...
2024-08-26,0.093281,-0.275848,0.275813
2024-08-27,-0.162765,0.111656,0.267059
2024-08-28,-0.126100,-0.507014,0.241756


# Split train, validation and test data

In [ ]:
def split_data(data, target, train_end =datetime(2022, 5, 30), test_start=datetime(2022, 5, 31), test_size=0.1):
  test_data = data.loc[test_start:]

  train_val = data.loc[:train_end]
  train_data, val_data = train_test_split(train_val, test_size=test_size, shuffle=False)

  return train_data, val_data, test_data

In [ ]:
def sliding_window_transform(X, y, timesteps):
  X_windows = []
  y_windows = []
  for i in range(len(X) - timesteps + 1):
    X_window = X[i:i + timesteps]
    y_window = y[i + timesteps - 1]
    X_windows.append(X_window)
    y_windows.append(y_window)

  return np.array(X_windows), np.array(y_windows)

In [ ]:
# Define the rolling window function
def rolling_window(df, in_sample_window_size, out_of_sample_size):
  X, y = [], []
  for i in range(in_sample_window_size, len(df) - out_of_sample_size):
    X.append(df.iloc[i - in_sample_window_size:i, :-1].values)  # All features except the target column
    y.append(df.iloc[i:i + out_of_sample_size, -1].values)  # Target column

  return np.array(X), np.array(y)

# Model

In [ ]:
def calculate_mape(actual, predicted):
  actual, predicted = np.array(actual), np.array(predicted)
  return np.mean(np.abs((actual - predicted) / actual)) * 100

## RSV

In [ ]:
daily_train, daily_val, daily_test = split_data(data, 'RSV')

In [ ]:
in_sam_win_sz = 25
out_sam_win_sz = 5

In [ ]:
RSV_X_train, RSV_y_train = rolling_window(daily_train, in_sam_win_sz, out_sam_win_sz)
RSV_X_val, RSV_y_val = rolling_window(daily_val, in_sam_win_sz, out_sam_win_sz)
RSV_X_test, RSV_y_test = rolling_window(daily_test, in_sam_win_sz, out_sam_win_sz)

In [ ]:
# RSV_X_train = RSV_X_train.reshape(RSV_X_train.shape[0], win_sz, -1)
# RSV_X_val = RSV_X_val.reshape(RSV_X_val.shape[0], win_sz, -1)
# RSV_X_test = RSV_X_test.reshape(RSV_X_test.shape[0], win_sz, -1)

### LSTM

In [ ]:
lstm_garch_model = Sequential()
lstm_garch_model.add(LSTM(48, input_shape=(RSV_X_train.shape[1], RSV_X_train.shape[2]), return_sequences=True))
lstm_garch_model.add(Dropout(0.1))
lstm_garch_model.add(LSTM(16))
lstm_garch_model.add(Dropout(0.1))
lstm_garch_model.add(Dense(5))

lstm_garch_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
lstm_garch_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 25, 48)              │           9,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 25, 48)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 16)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 5)                   │              85 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,037 (54.83 KB)

 Trainable params: 14,037 (54.83 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
lstm_history = lstm_garch_model.fit(RSV_X_train, RSV_y_train, epochs=80, batch_size=32, validation_data=(RSV_X_val, RSV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 22ms/step - loss: 0.2943 - val_loss: 0.1405
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 27ms/step - loss: 0.1265 - val_loss: 0.1697
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 0.1408 - val_loss: 0.2051
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 19ms/step - loss: 0.1588 - val_loss: 0.1792
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 26ms/step - loss: 0.0992 - val_loss: 0.1547
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - loss: 0.1026 - val_loss: 0.1412
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 0.0941 - val_loss: 0.2165
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 20ms/step - loss: 0.0717 - val_loss: 0.1947
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - loss: 0.0657 - val_loss: 0.2200
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 22ms/step - loss: 0.0581 - val_loss: 0.1828
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - loss: 0.0530 - val_loss: 0.1956
Epoch 12/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
# Predictions
RSV_test_predictions = lstm_garch_model.predict(RSV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step


In [ ]:
mse = mean_squared_error(RSV_y_test, RSV_test_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(RSV_y_test, RSV_test_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(RSV_y_test, RSV_test_predictions)
print(f'MAPE: {mape}%')

MSE: 0.09465983160083064
RMSE: 0.30766837926707813
MAE: 0.22480048037098938
MAPE: 225.06861488571474%


### GRU

In [ ]:
gru_model = Sequential()
gru_model.add(GRU(48, input_shape=(RSV_X_train.shape[1], RSV_X_train.shape[2]), return_sequences=True))
gru_model.add(Dropout(0.1))
gru_model.add(GRU(16))
gru_model.add(Dropout(0.1))
gru_model.add(Dense(5))

gru_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
gru_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru (GRU)                            │ (None, 25, 48)              │           7,488 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 25, 48)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_1 (GRU)                          │ (None, 16)                  │           3,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 5)                   │              85 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 10,741 (41.96 KB)

 Trainable params: 10,741 (41.96 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
gru_history = gru_model.fit(RSV_X_train, RSV_y_train, epochs=80, batch_size=32, validation_data=(RSV_X_val, RSV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 27ms/step - loss: 0.2655 - val_loss: 0.1362
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - loss: 0.1341 - val_loss: 0.1313
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 25ms/step - loss: 0.1165 - val_loss: 0.2081
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - loss: 0.0978 - val_loss: 0.1811
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - loss: 0.0880 - val_loss: 0.1782
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - loss: 0.0676 - val_loss: 0.2748
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - loss: 0.0662 - val_loss: 0.2248
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - loss: 0.0578 - val_loss: 0.2693
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - loss: 0.0603 - val_loss: 0.1572
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - loss: 0.0484 - val_loss: 0.1759
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 33ms/step - loss: 0.0551 - val_loss: 0.2598
Epoch 12/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 

In [34]:
# Predictions
RSV_gru_predictions = gru_model.predict(RSV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step


In [35]:
mse = mean_squared_error(RSV_y_test, RSV_gru_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(RSV_y_test, RSV_gru_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(RSV_y_test, RSV_gru_predictions)
print(f'MAPE: {mape}%')

MSE: 0.05273355542794069
RMSE: 0.2296378789048982
MAE: 0.1829500366001511
MAPE: 129.9998681668044%


### LSTM_GRU

In [36]:
lstm_gru_model = Sequential()
lstm_gru_model.add(LSTM(48, input_shape=(RSV_X_train.shape[1], RSV_X_train.shape[2]), return_sequences=True))
lstm_gru_model.add(Dropout(0.1))
lstm_gru_model.add(GRU(16))
lstm_gru_model.add(Dropout(0.1))
lstm_gru_model.add(Dense(5))

lstm_gru_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [37]:
lstm_gru_model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_3 (LSTM)                        │ (None, 25, 48)              │           9,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 25, 48)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_3 (GRU)                          │ (None, 16)                  │           3,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 5)                   │              85 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 13,045 (50.96 KB)

 Trainable params: 13,045 (50.96 KB)

 Non-trainable params: 0 (0.00 B)

In [38]:
lstm_gru_history = lstm_garch_model.fit(RSV_X_train, RSV_y_train, epochs=80, batch_size=32, validation_data=(RSV_X_val, RSV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - loss: 0.0177 - val_loss: 0.2173
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - loss: 0.0179 - val_loss: 0.2083
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 0.0176 - val_loss: 0.2220
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - loss: 0.0172 - val_loss: 0.2293
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 22ms/step - loss: 0.0176 - val_loss: 0.2234
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 0.0170 - val_loss: 0.1880
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - loss: 0.0163 - val_loss: 0.2139
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step - loss: 0.0164 - val_loss: 0.1982
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - loss: 0.0459 - val_loss: 0.1858
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/step - loss: 0.0322 - val_loss: 0.2323
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - loss: 0.0261 - val_loss: 0.2131
Epoch 12/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 

In [39]:
# Predictions
RSV_lstm_gru_predictions = lstm_garch_model.predict(RSV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


In [40]:
mse = mean_squared_error(RSV_y_test, RSV_lstm_gru_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(RSV_y_test, RSV_lstm_gru_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(RSV_y_test, RSV_lstm_gru_predictions)
print(f'MAPE: {mape}%')

MSE: 0.09635601850368629
RMSE: 0.31041265841406385
MAE: 0.22380264699453792
MAPE: 370.42519704259104%
